In [1]:
from os import path
import re
from zipfile import ZipFile

xml_tag_re = re.compile(r'</?DOC|</?TEXT')


def yield_ja_textbook_zip_lines(zip_file_path, inner_folder_path=None):
    subtree = path.splitext(path.basename(zip_file_path))[0]
    if inner_folder_path:
        subtree = path.join(subtree, inner_folder_path)
    with ZipFile(zip_file_path) as z:
        namelist = z.namelist()
        names = list(filter(lambda x: path.commonpath([subtree, x]) == subtree, namelist))
        for filename in names:
            info = z.getinfo(filename)
            if info.is_dir():
                continue
            with z.open(filename) as f:
                for line in f:
                    line = line.decode('UTF-8').strip()
                    if xml_tag_re.match(line):
                        continue
                    if line.startswith('<Title>'):
                        line = line.replace('<Title>', '').replace('</Title>', '')
                    yield line

In [2]:
# from joblib import Parallel, delayed
import MeCab


mecab = MeCab.Tagger('-Owakati')


def get_tokenized_line_ja(line):
    return mecab.parse(line).strip()


def yield_tokenized_lines_ja(lines):
    # return Parallel(n_jobs=-3)(delayed(get_tokenized_line_ja)(line) for line in lines)
    for line in lines:
        yield get_tokenized_line_ja(line).split()

In [3]:
from gensim.models.word2vec import Word2Vec


def get_w2v(lines):
    return Word2Vec(sentences=lines, min_count=1, workers=8)

In [4]:
tokyoshoseki_generator = yield_ja_textbook_zip_lines(
    'qalab3-essay-phase2/_references/training_set/qalab3-ja-knowledgesource/'
    'qalab3-ja-knowledgesource/make-index-Tokyoshoseki.zip',
    'tokyoshoseki-newxml')
tokenized_tokyoshoseki = yield_tokenized_lines_ja(tokyoshoseki_generator)

In [5]:
yamakawa_generator = yield_ja_textbook_zip_lines(
    'qalab3-essay-phase2/_references/training_set/qalab3-ja-knowledgesource/'
    'qalab3-ja-knowledgesource/make-index-Yamakawa.zip',
    'yamakawa-Sekaishi-newxml')
tokenized_yamakawa = yield_tokenized_lines_ja(yamakawa_generator)

In [ ]:
# jawiki_generator = yield_ja_textbook_zip_lines(
#     'qalab3-essay-phase2/_references/training_set/jawiki-100M-newxml.zip')
# tokenized_jawiki = yield_tokenized_lines_ja(jawiki_generator)

Process ForkPoolWorker-21:
Process ForkPoolWorker-23:
Process ForkPoolWorker-19:
Process ForkPoolWorker-24:
Process ForkPoolWorker-20:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.

In [6]:
from itertools import chain


line_list_of_token_list = chain(tokenized_tokyoshoseki, tokenized_yamakawa)

In [7]:
model_ja = get_w2v(line_list_of_token_list)

In [8]:
model_ja.wv.vocab

{'さまざま': <gensim.models.keyedvectors.Vocab at 0x10613e358>,
 'な': <gensim.models.keyedvectors.Vocab at 0x10613e438>,
 '世界': <gensim.models.keyedvectors.Vocab at 0x10613e320>,
 'の': <gensim.models.keyedvectors.Vocab at 0x10613e4e0>,
 '時代': <gensim.models.keyedvectors.Vocab at 0x10613e3c8>,
 'いろいろ': <gensim.models.keyedvectors.Vocab at 0x10c945828>,
 '自然': <gensim.models.keyedvectors.Vocab at 0x10c9456a0>,
 '環境': <gensim.models.keyedvectors.Vocab at 0x10c9456d8>,
 'もと': <gensim.models.keyedvectors.Vocab at 0x10c945710>,
 'で': <gensim.models.keyedvectors.Vocab at 0x10c945748>,
 '，': <gensim.models.keyedvectors.Vocab at 0x10c945860>,
 '人々': <gensim.models.keyedvectors.Vocab at 0x10c945898>,
 'は': <gensim.models.keyedvectors.Vocab at 0x10c9458d0>,
 'その': <gensim.models.keyedvectors.Vocab at 0x10c945908>,
 '地': <gensim.models.keyedvectors.Vocab at 0x10c945940>,
 '風土': <gensim.models.keyedvectors.Vocab at 0x10c945978>,
 'に': <gensim.models.keyedvectors.Vocab at 0x10c9459b0>,
 '根ざし': <gensim.m

In [9]:
model_ja.wv.most_similar(['東アジア'])

[('排出', 0.39770206809043884),
 ('置', 0.3571590185165405),
 ('法則', 0.34622806310653687),
 ('マカオ', 0.34412676095962524),
 ('離婚', 0.3395390212535858),
 ('にまつわる', 0.3390005826950073),
 ('以来', 0.3386876881122589),
 ('車軸', 0.3382636308670044),
 ('きわめ', 0.32996922731399536),
 ('ネルヴァ', 0.32973313331604004)]

In [10]:
model_ja.init_sims(replace=True)

In [ ]:
summary = 'カイロの中心は、ナイル川東岸にほど近いタハリール広場である。タハリール広場の周辺にはモガンマアと呼ばれる政府庁舎ビルや国民民主党本部、カイロ・アメリカン大学のキャンパスやエジプト考古学博物館、アラブ連盟本部ビルなどがあり、またカイロ地下鉄のターミナルであるサダト駅もこの地下にある。2011年エジプト騒乱においては、この広場に民衆が集結し、ホスニー・ムバーラク大統領の退陣を求める民主化デモの拠点となった。このあたりはカスル・アンニール地区と呼ばれる。タハリール広場から北東にはタラアト・ハルプ広場があり、この周辺からタラアト・ハルプ通りやカスル・アンニール通りにかけての一帯が繁華街となっている。その北東にはオスマン時代には湖であったところをムハンマド・アリー朝時代に埋め立てて作ったイズベキーヤ公園がある。さらにその北にはカイロの中央駅であるラムセス駅があり、ここからアレキサンドリアやデルタ各地、アスワンなど上エジプトに列車が走っている。また、タハリール広場から東に400mほどのところにムハンマド・アリー朝の王宮であり、現大統領府であるアブディーン宮殿がある。タハリール広場から南のナイル川沿いはガーデン・シティと呼ばれ、イギリス統治時代にエジプト総督府がおかれ開発が進められたエリアである。ガーデン・シティの名の通り、庭園に囲まれた高級住宅街が広がっている。このタハリール広場を中心とした地'

In [ ]:
tokenized_summary = get_tokenized_line_ja(summary)

In [ ]:
gold = '古代エジプトは、ナイル川を中心に長らく独立王朝が栄えたが、アレクサンドロス大王などの征服を受ける。プトレマイオス朝エジプトのクレオパトラは、地中海の覇権国であったローマの内乱に際し、アントニウスと連合してオクタヴィアヌスに対抗するが、アクティウムの海戦に敗れ、エジプトはローマの属州となった。紀元7世紀にアラビア半島を統一したイスラム教勢力は、東ローマ帝国とササン朝ペルシアの対立に乗じて版図を拡大し、エジプトを征服した。イスラム帝国の版図の一部を継承したファーティマ朝では、1196年にサラディンが宰相となり、イスラム教勢力から聖地エルサレムを奪還しようとする十字軍に対抗した。同じく、十字軍に対抗する中でエジプトに建てられたマムルーク朝は、オスマン帝国の侵攻を受けて滅ぶ。近代には、イギリス本国とインドとの連絡線を断つために、フランスのナポレオンがエジプトに進攻し、解放者を称するが、民衆の抵抗に遭った。対仏戦争に際して、オスマン帝国がエジプトに派遣したムハンマド・アリーは、同地に王朝を建てる。このムハンマド・アリー朝は、イギリスによる内政への介入を受けるようになり、その保護国となる。ナセルは、1952年に革命を起こして王政を倒し、イスラエルとの戦争を指導した。'

In [ ]:
tokenized_gold = get_tokenized_line_ja(gold)

In [ ]:
model_ja.wmdistance(tokenized_summary.split(), tokenized_gold.split())

In [11]:
model_ja.save('w2v_ja.model')

In [4]:
import os


def listdir_file_path(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

In [5]:
def yield_tokenized_en_wiki_lines(folder_path='qalab3-essay-phase2/enwiki_doc_by_id-tokenized'):
    for file_path in listdir_file_path(folder_path):
        with open(file_path) as f:
            for line in f:
                line = line.strip()
                if xml_tag_re.match(line):
                    continue
                if line.startswith('<Title>'):
                    line = line.replace('<Title>', '').replace('</Title>', '')
                yield line.split()

In [6]:
from itertools import chain


line_list_of_en_token_list = chain(yield_tokenized_en_wiki_lines())

In [7]:
model_en = get_w2v(line_list_of_en_token_list)

In [8]:
model_en.wv.vocab

{'Emil': <gensim.models.keyedvectors.Vocab at 0x110cee1d0>,
 'Kraepelin': <gensim.models.keyedvectors.Vocab at 0x10ecf7438>,
 '-LCB-': <gensim.models.keyedvectors.Vocab at 0x115789940>,
 'Infobox': <gensim.models.keyedvectors.Vocab at 0x115789a20>,
 'scientist': <gensim.models.keyedvectors.Vocab at 0x1157890f0>,
 '|': <gensim.models.keyedvectors.Vocab at 0x115789240>,
 'name': <gensim.models.keyedvectors.Vocab at 0x115789908>,
 '=': <gensim.models.keyedvectors.Vocab at 0x115789a90>,
 'image': <gensim.models.keyedvectors.Vocab at 0x115789080>,
 '1926.jpg': <gensim.models.keyedvectors.Vocab at 0x115789358>,
 'image_size': <gensim.models.keyedvectors.Vocab at 0x115789710>,
 'alt': <gensim.models.keyedvectors.Vocab at 0x1157891d0>,
 'caption': <gensim.models.keyedvectors.Vocab at 0x115789320>,
 'in': <gensim.models.keyedvectors.Vocab at 0x115789780>,
 'his': <gensim.models.keyedvectors.Vocab at 0x1157898d0>,
 'later': <gensim.models.keyedvectors.Vocab at 0x115789048>,
 'years': <gensim.mod

In [10]:
model_en.wv.most_similar(['German'])

[('Ultramarinas', 0.46172091364860535),
 ('Pepe', 0.44785213470458984),
 ('vice-principal', 0.4434520900249481),
 ('transmit/receive', 0.4318106472492218),
 ('Anushtubh', 0.42985594272613525),
 ('Baccalaureate', 0.42230531573295593),
 ('范陽王', 0.41674861311912537),
 ('pumped-out', 0.41333553194999695),
 ('http://www.cntraveler.com/readers-choice/2011/europe/top-10-islands-europe#slide=1',
  0.4023128151893616),
 ('Announcers', 0.4007004499435425)]

In [11]:
model_en.init_sims(replace=True)

In [12]:
model_en.save('w2v_en.model')